In [28]:
## ::: NOTE ::: this script requires the R packages 'stringr' and 'DESeq2'. Please download them into the environment from which you will run this script.

##if these packages are not installed, uncomment the next two lines
#BiocManager::install("stringr", force = TRUE)
#BiocManager::install("DESeq2", force = TRUE)

library(DESeq2)
library(stringr)

In [68]:
##read in the 'gene_count_matrix.csv' file we created from the make-deseq-reable-files.qsub script
countData <- as.matrix(read.csv("gene_count_matrix.csv", row.names="gene_id"))

In [69]:
##because the columns are currently named after the folder and not the sample, we will correct that manually
colnames(countData)<-c("SRR10416456","SRR10416457","SRR10416458","SRR10416459","SRR10416460","SRR10416461","SRR10416462","SRR10416463","SRR10416464","SRR10416465","SRR10416466","SRR10416467")
head(countData)

,SRR10416456,SRR10416457,SRR10416458,SRR10416459,SRR10416460,SRR10416461,SRR10416462,SRR10416463,SRR10416464,SRR10416465,SRR10416466,SRR10416467
MSTRG.30,0,0,0,155,0,0,0,0,829,167,202,0
MSTRG.4,340,2128,2,202,375,2,81,490,166,660,1416,1338
MSTRG.31,585,1434,5,976,292,336,0,436,1271,2155,508,1842
MSTRG.7,795,187,1,1079,152,160,1915,470,882,992,1616,626
MSTRG.28,1105,412,83,486,1250,822,293,1794,1934,2990,622,2508
MSTRG.5,520,0,17,60,0,0,0,169,285,420,69,120


In [77]:
##each sample will be given a unqiue tag corresponding to its experimental design
##the format is treatmen.timepoint.replicate
##these are manually currated based on the name of the sample on NCBI 'https://www.ncbi.nlm.nih.gov/sra?LinkName=bioproject_sra_all&from_uid=588343'
sample<-c("Reseeded.24h.1", ##SRR10416456
             "Reseeded.05h.3", ##SRR10416457
             "Reseeded.05h.2", ##SRR10416458
             "Reseeded.05h.1", ##SRR10416459
             "Diatom_only.24h.3", ##SRR10416460
             "Diatom_only.24h.2", ##SRR10416461
             "Diatom_only.24h.1", ##SRR10416462
             "Diatom_only.05h.3", ##SRR10416463
             "Reseeded.24h.3", ##SRR10416464
             "Reseeded.24h.2", ##SRR10416465
             "Diatom_only.05h.2", ##SRR10416466
             "Diatom_only.05h.1") ##SRR10416467

##now we will split this tag up into each of its three parts
splits<-str_split(sample,fixed("."))
treatment<-sapply(splits,"[[",1)
time<-sapply(splits,"[[",2)
replicate<-sapply(splits,"[[",3)

##and we will use this data to create a matrix that contains all of this metadata for each sample
matrix.data <- c(treatment,time,replicate)
colData<-matrix(matrix.data,nrow=length(sample),ncol=3,byrow=FALSE)
colnames(colData)<-c("treatment","time","replicate")
##the datatable colData should be a 12 row matrix (one for each sample) with three columns 'treatment' (reseeded or diatom only), 'time' (time point collected), and replicate (biological replicate: 1 to 3)
##each sample should have a unique combination of these three parameters


rownames(colData)<-colnames(countData)
##now we rename columns of the matrix so they are identical to our countData datatable (as is required by DESeq)

            treatment     time  replicate
SRR10416456 "Reseeded"    "24h" "1"      
SRR10416457 "Reseeded"    "05h" "3"      
SRR10416458 "Reseeded"    "05h" "2"      
SRR10416459 "Reseeded"    "05h" "1"      
SRR10416460 "Diatom_only" "24h" "3"      
SRR10416461 "Diatom_only" "24h" "2"      
SRR10416462 "Diatom_only" "24h" "1"      
SRR10416463 "Diatom_only" "05h" "3"      
SRR10416464 "Reseeded"    "24h" "3"      
SRR10416465 "Reseeded"    "24h" "2"      
SRR10416466 "Diatom_only" "05h" "2"      
SRR10416467 "Diatom_only" "05h" "1"      


In [79]:
##we will now check whether our tables are formatted in a way DESeq asks for, as suggested by the StringTie manual (http://ccb.jhu.edu/software/stringtie/index.shtml?t=manual)
all(rownames(colData) %in% colnames(countData))
countData <- countData[, rownames(colData)]
all(rownames(colData) == colnames(countData))
##both should return TRUE

[1] TRUE

[1] TRUE

In [95]:
##now, as is done in Shibl et al. 2020, we will divide the data into two different comparisions
##one between the reseeded culture, and the diatom alone at 0.5hrs
##and the other between the reseeded culture, and the diatom alone at 24hrs
first.timepoints<-colData[which(colData[,"time"]=='24h'),]
first.timepoint.counts<-countData[,which(colnames(countData) %in% rownames(first.timepoints))]

endpoints<-colData[which(colData[,"time"]=='05h'),]
endpoint.counts<-countData[,which(colnames(countData) %in% rownames(endpoints))]


##and we'll use this subsetted data to form a DESeq dataset
##the design = ~ treatment flag signifies we want to compare expression between the treatment variables
##because of the way DESeq works, the diatom_alone treatment is automatically our control because it is first alphametically
comparing.05h <- DESeqDataSetFromMatrix(countData = first.timepoint.counts,
        colData = first.timepoints, design = ~ treatment)

comparing.24h <- DESeqDataSetFromMatrix(countData = endpoint.counts,
        colData = endpoints, design = ~ treatment)


#we will also still analyze the data from the 4-way treatment comparision as well
#'design = ~ time + treatment' says we are still interested in a treatment comparision, but controlled for timepoint
overallcomparision <- DESeqDataSetFromMatrix(countData = countData,
        colData = colData, design = ~ time + treatment)

In [104]:
##now for each DESeq dataset we created, we will run a DESeq differential expression analysis on this, and save the results into a file
##the files are : res.24 (for the 24 hour comparision), res.05 (for the 0.5hr comparision), and res.overall (for a time controlled comparision of every sample)

dds.24 <- DESeq(comparing.24h)
res.24 <- results(dds.24)
dds.05 <- DESeq(comparing.05h)
res.05 <- results(dds.05)
dds.overall <- DESeq(overallcomparision)
res.overall <- results(dds.overall)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [112]:
#we now write the results of this analysis to csvs that will be readble for our figuring making in our 'figure1b-circos' script
write.table(res.24,file="firstimepoints.diff.ex.comparison.csv",sep=",")
write.table(res.05,file="endpoint.diff.ex.comparison.csv",sep=",")
write.table(res.overall,file="overall.diff.ex.comparison.csv",sep=",")